In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

### Prepare train and test data



In [ ]:
### Extract features


In [ ]:
### Normalize
from sklearn.preprocessing import StandardScaler

X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

In [ ]:
### Classifier
import time
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

# Create a linear SVM as classifier
svc = LinearSVC()
# Check the training time for the SVC
t = time.time()
svc.fit( X_train, y_train )
t2 = time.time()
print( round( t2 - t, 2 ), 'Seconds to train SVC...' )
